In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
from debiaswe.data import load_gender_seed

import torch

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
# E = WordEmbedding('./embeddings/GoogleNews-vectors-negative300.bin') # Not possible
print(E)
words = E.words
print("Words:", len(words))

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Words: 26423
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# gender direction
v_gender = E.diff('she', 'he')

# Uncomment below for direction based on multiple definitional pairs
# with open('./data/definitional_pairs.json', "r") as f:
#     defs = json.load(f)
# v_gender = we.doPCA(defs, E).components_[0]

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, thresh=1)

# for (a,b,c) in a_gender:
#     print(a+"-"+b)
we.viz(a_gender)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
(135018, 300)
   0                          she | he                           1.00
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                        woman | man                          0.75
   4                     daughter | son                          0.67
   5                businesswoman | businessman                  0.65
   6                         girl | boy                          0.65
   7                      actress | actor                        0.65
   8                   chairwoman | chairman                     0.63
   9                      heroine | hero                         0.62
  10                       mother | father                       0.60
  11                  spokeswoman | spokesman                    0.59
  12                       sister | brother                      0.59
  13               

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
# profession analysis gender
sp = E.profession_stereotypes(profession_words, v_gender)

                Female                |                 Male                 
-----------------------------------------------------------------------------
0.36                    businesswoman | maestro                        -0.238
0.352                         actress | statesman                      -0.217
0.34                        housewife | skipper                        -0.208
0.304                       homemaker | protege                        -0.203
0.304                registered_nurse | businessman                    -0.202
0.281                           nurse | sportsman                      -0.195
0.275                        waitress | philosopher                    -0.188
0.273                    receptionist | marksman                       -0.181
0.266                       librarian | captain                        -0.173
0.257                       socialite | architect                      -0.168
0.253                       ballerina | financier               

## Find soft debias transform using SGD
This code is largely based on code from https://github.com/TManzini/DebiasMulticlassWordEmbedding.

In [7]:
from debiaswe.debias import soft_debias

In [8]:
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)

In [9]:
soft_debias(E, gender_specific_words, defs)

Loss @ Epoch #0: 854547.5625
Loss @ Epoch #100: 173443.71875
Loss @ Epoch #200: 61013.32421875
Loss @ Epoch #300: 22842.662109375
Loss @ Epoch #400: 8393.376953125
Loss @ Epoch #500: 2771.1708984375
Loss @ Epoch #600: 658.6480712890625
Loss @ Epoch #700: 202.66058349609375
Loss @ Epoch #800: 120.29017639160156
Loss @ Epoch #900: 120.28070068359375
Loss @ Epoch #1000: 86.58453369140625
Loss @ Epoch #1100: 10.40377426147461
Loss @ Epoch #1200: 16.98859977722168
Loss @ Epoch #1300: 14.821828842163086
Loss @ Epoch #1400: 16.662677764892578
Loss @ Epoch #1500: 21.607131958007812
Loss @ Epoch #1600: 3.3599722385406494
Loss @ Epoch #1700: 3.3920795917510986
Loss @ Epoch #1800: 3.3595547676086426
Loss @ Epoch #1900: 2.2684056758880615
Lowest loss: 2.2181766033172607


In [10]:
# save soft-debiased embeddings

E.save('./embeddings/w2v_gnews_small_soft_debiased.txt')
# E.save('./embeddings/GoogleNews-vectors-negative300_soft_debiased.bin')

Wrote 26423 words to ./embeddings/w2v_gnews_small_soft_debiased.txt


In [11]:
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)
we.viz(a_gender_debiased)

   0                          she | he                           1.00
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                        woman | man                          0.75
   4                     daughter | son                          0.67
   5                businesswoman | businessman                  0.65
   6                         girl | boy                          0.65
   7                      actress | actor                        0.65
   8                   chairwoman | chairman                     0.63
   9                      heroine | hero                         0.62
  10                       mother | father                       0.60
  11                  spokeswoman | spokesman                    0.59
  12                       sister | brother                      0.59
  13                        girls | boys                         0.59
  14                

In [12]:
# profession analysis gender
sp = E.profession_stereotypes(profession_words, v_gender)

                Female                |                 Male                 
-----------------------------------------------------------------------------
0.126                       caretaker | philosopher                    -0.149
0.124                     firefighter | sociologist                    -0.103
0.121                         fireman | landlord                       -0.102
0.116                        teenager | associate_dean                 -0.101
0.109                       lifeguard | psychologist                   -0.099
0.099                       patrolman | psychiatrist                   -0.098
0.099                freelance_writer | lawyer                         -0.095
0.097                     taxi_driver | playwright                      -0.09
0.094                      naturalist | lecturer                       -0.087
0.093                            monk | attorney                       -0.085
0.092                             nun | parliamentarian         